In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import  pickle

In [2]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/Tetris-v5")

Action Space: Discrete(5)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [3]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 12
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 9
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 5
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  #count_squares = 2
  #for i in range(count_squares):
  #  for j in range(count_squares):
  #    x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
  #    y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [13]:
n_actions = 5
embed_size = 250#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open('./genoms/best_genom_tetris.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

loaded successfully


In [5]:
#3, 1, 4, 4

In [14]:
n_actions = 5
embed_size = 100800#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(tacts=1,input_size=embed_size,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [7]:
def test_atary(genom, draw=False,game_name="ALE/Tetris-v5",seed=0):

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  #np.random.seed(seed)
  #env.seed(seed)
  while True:
  #for i in range(200):
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
        
      shp = np.shape(state)
      #print(shp)
      #1/0
      state = np.reshape(state,[shp[2],shp[0],shp[1]])
      #print(np.shape(state))
        
      t=pd.Timestamp.now()
      if done:
        break
      #state = encode_mobnet(np.array([state]))
      #state = encode_simple(state)
      reward_sum += (reward + 0.1)
      out_tape = controller.act(state,reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum

In [8]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/Tetris-v5"
  #один энвайронмент, разные сиды
  q_arr = []
  for seed in range(3):
    q_arr.append(test_atary(genom, draw=draw,game_name=game_name))
  return np.sum(q_arr)/len(q_arr)

In [9]:
#(210, 160, 3)

In [10]:
%%time
q=test_atary(genom,game_name="ALE/Tetris-v5", draw=False)
print('q=',q)

q= 8.499999999999986
CPU times: user 5.18 s, sys: 292 µs, total: 5.18 s
Wall time: 961 ms


In [12]:
# signal 26.2 s
# for for 4min 12s

In [11]:
%%time
q=multy_test(genom)
print('q=',q)
#q=multy_test(genom)
#print('q=',q)

q= 8.499999999999986
CPU times: user 16.1 s, sys: 5.96 ms, total: 16.1 s
Wall time: 2.96 s


In [13]:
%%time
q=test_atary(genom)
print('q=',q)

q= 20.000000000000014
CPU times: user 12 s, sys: 4.38 ms, total: 12 s
Wall time: 2.09 s


In [ ]:
print(pd.Timestamp.now())

In [15]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=1,init_file='genoms/genom_tetris1.pkl',history_file='history/history_tetris1.pkl')

1
history loaded successfully


In [ ]:
try:
    with open('genoms/best_genom_tetris.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

In [17]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_tetris.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
random trial
scores for optimizers augmented [1.28185558e+00 1.56771923e+00 2.75769159e+00 2.09297577e+00
 5.45192113e+00 2.93160352e+00 3.52325083e+00 9.16199394e+01
 1.00000000e+10]
chosen evol_narrow previous_result: nan per tacts: nan
iteration 0 y= [58.1        50.76666667]
iteration 1 y= [84.1        61.83333333]
iteration 2 y= [78.4 69.7]
iteration 3 y= [77.7 76.9]
iteration 4 y= [78.36666667 74.76666667]
iteration 5 y= [80.76666667 75.03333333]
iteration 6 y= [71.96666667 69.3       ]
iteration 7 y= [79.7        77.56666667]
iteration 8 y= [79.56666667 79.43333333]
iteration 9 y= [77.96666667 77.43333333]
iteration 10 y= [79.7        77.83333333]
iteration 11 y= [83.43333333 83.16666667]
iteration 12 y= [80.06666667 79.56666667]
iteration 13 y= [80.23333333 76.23333333]
iteration 14 y= [75.83333333 75.03333333]
iteration 15 y= [82.63333333 81.3       ]
iteration final y= [76.23333333 76.23333333]
result evol_narrow previous_gain: 18.133333333332722 per tacts: 136 duratio

In [ ]:
with open('./genoms/genom_tetris1.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]

In [16]:
q=multy_test(genom,draw=True)
print('q=',q)

q= 20.000000000000014


In [17]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [ ]:
#noise stategy
#with open('genoms/best_genom_tetris.pkl', 'rb') as f:
    #genom = pickle.load(f)[-1]
globals()['video'] = []
noise = np.random.normal(size=[len(genom)])*1
for i in range(1):
    genom = genom+noise
    q=test_atary(genom, draw=True,game_name="ALE/Tetris-v5",seed=0)
    np.random.seed(i)
    noise = np.random.normal(size=[len(genom)])*1



In [ ]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=10,
    loop=0
)

In [ ]:
with open('genoms/best_genom_tetris.pkl', 'wb') as f:
    pickle.dump(genom,f,protocol=pickle.HIGHEST_PROTOCOL)
    print('WRITTEN')